# Zonal stats at scale

We will now use the GLAD LCLU data, and calculate per-country statistics using GADM Level 0 boundaries.

To save time, we will do this only for South America

In [7]:
# bounding box for south america

SA_BBOX = (-92, -56, -30, 12)

## Read datacube

In [1]:
import icechunk as ic
import xarray as xr

# Configure the Icechunk storage backend to read from the public S3 bucket
storage = ic.s3_storage(
    bucket="icechunk-public-data",
    prefix=f"v1/glad",
    region="us-east-1",
    anonymous=True,
)
# Open the Icechunk repository and create a read-only session
repo = ic.Repository.open(storage=storage)
session = repo.readonly_session("main")
ds = xr.open_dataset(
    session.store, chunks=None, consolidated=False, engine="zarr"
).chunk({"x": 5000, "y": 5000})
ds

<xarray.Dataset> Size: 4TB
Dimensions:      (y: 560000, x: 1440000, year: 5)
Coordinates:
    spatial_ref  int64 8B ...
  * y            (y) float64 4MB 80.0 80.0 80.0 80.0 ... -60.0 -60.0 -60.0 -60.0
  * x            (x) float64 12MB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * year         (year) int64 40B 2000 2005 2010 2015 2020
Data variables:
    lclu         (year, y, x) uint8 4TB dask.array<chunksize=(5, 5000, 5000), meta=np.ndarray>
Attributes:
    license:  CC BY 4.0

## Read GADM Geometries

I have converted these to geoparquet.

In [20]:
import geopandas as gpd
import dask_geopandas as dgpd

gadm_pq_uri = f"s3://earthmover-scratch/gadm/partitioned_2/ADM_0.parquet"
geoms = gpd.read_parquet(
    gadm_pq_uri,
    #calculate_divisions=True,
    #split_row_groups=True,
)
geoms

,GID_0,COUNTRY,geometry
__null_dask_index__,,,
0,ABW,Aruba,"MULTIPOLYGON (((-69.9782 12.46986, -69.97847 1..."
1,AFG,Afghanistan,"MULTIPOLYGON (((63.61554 29.4697, 63.61425 29...."
2,AGO,Angola,"MULTIPOLYGON (((19.89892 -17.87674, 19.89082 -..."
3,AIA,Anguilla,"MULTIPOLYGON (((-63.02064 18.2075, -63.02587 1..."
4,ALA,Åland,"MULTIPOLYGON (((21.32306 59.74847, 21.32306 59..."
...,...,...,...
258,YEM,Yemen,"MULTIPOLYGON (((44.12514 12.63236, 44.12542 12..."
259,ZAF,South Africa,"MULTIPOLYGON (((19.66291 -34.78653, 19.66327 -..."
260,ZMB,Zambia,"MULTIPOLYGON (((25.87834 -17.97218, 25.87034 -..."


# Rasterize the geometries



In [21]:
from rasterix.rasterize.rasterio import rasterize

In [25]:
south_america = ds.rio.clip_box(*SA_BBOX)
clipped_geoms = geoms.loc[~geoms.clip_by_rect(*SA_BBOX).is_empty].reset_index(drop=True)

In [26]:
south_america.coords["adm_0"] = rasterize(south_america, clipped_geoms[["geometry"]])
south_america

<xarray.Dataset> Size: 405GB
Dimensions:      (y: 272000, x: 248000, year: 5)
Coordinates:
  * y            (y) float64 2MB 12.0 12.0 12.0 12.0 ... -56.0 -56.0 -56.0 -56.0
  * x            (x) float64 2MB -92.0 -92.0 -92.0 -92.0 ... -30.0 -30.0 -30.0
  * year         (year) int64 40B 2000 2005 2010 2015 2020
    spatial_ref  int64 8B 0
    adm_0        (y, x) uint8 67GB dask.array<chunksize=(3000, 3000), meta=np.ndarray>
Data variables:
    lclu         (year, y, x) uint8 337GB dask.array<chunksize=(5, 3000, 3000), meta=np.ndarray>
Attributes:
    license:  CC BY 4.0

## Set up the LULC grouper

In [27]:
import pandas as pd
from xarray.groupers import BinGrouper

legend = {
    "Terra Firma True desert": pd.Interval(0, 1, closed="both"),
    "Terra Firma Semi-arid": pd.Interval(2, 18, closed="both"),
    "Terra Firma Dense short vegetation": pd.Interval(19, 24, closed="both"),
    "Terra Firma Tree cover": pd.Interval(25, 48, closed="both"),
    "Wetland Salt Pan": pd.Interval(100, 101, closed="both"),
    "Wetland Sparse Vegetation": pd.Interval(102, 118, closed="both"),
    "Wetland Dense short Vegetation": pd.Interval(119, 124, closed="both"),
    "Wetland Tree Cover": pd.Interval(125, 148, closed="both"),
    "Open surface water": pd.Interval(200, 207, closed="both"),
    "Snow/Ice": pd.Interval(241, 241, closed="both"),
    "Cropland": pd.Interval(244, 244, closed="both"),
    "Built-up": pd.Interval(250, 250, closed="both"),
    "Ocean": pd.Interval(254, 254, closed="both"),
}

index = pd.IntervalIndex(list(legend.values()))

land_grouper = BinGrouper(bins=index, labels=list(legend.keys()))

## Set up computation

In [42]:
import numpy as np
from xarray.groupers import UniqueGrouper

pixel_counts = south_america.groupby(
    lclu=land_grouper,
    adm_0=UniqueGrouper(labels=np.arange(len(clipped_geoms))),
).count(dim=("y", "x"))
pixel_counts["adm_0"] = ("adm_0", clipped_geoms["GID_0"])
pixel_counts

<xarray.Dataset> Size: 11kB
Dimensions:      (year: 5, lclu_bins: 13, adm_0: 21)
Coordinates:
  * year         (year) int64 40B 2000 2005 2010 2015 2020
    spatial_ref  int64 8B 0
  * lclu_bins    (lclu_bins) object 104B 'Terra Firma True desert' ... 'Ocean'
  * adm_0        (adm_0) object 168B 'ARG' 'BOL' 'BRA' ... 'TTO' 'URY' 'VEN'
Data variables:
    lclu         (year, lclu_bins, adm_0) float64 11kB dask.array<chunksize=(5, 13, 21), meta=np.ndarray>
Attributes:
    license:  CC BY 4.0

In [40]:
import coiled
from distributed import Client

cluster = coiled.Cluster(n_workers=(10, 200), worker_options={"nthreads": 2})
client = Client(cluster)

Output()

╭──────────────────────────────── Package Info ────────────────────────────────╮
│           ╷                                                                  │
│   Package │ Note                                                             │
│ ╶─────────┼────────────────────────────────────────────────────────────────╴ │
│   xarray  │ Wheel built from                                                 │
│           │ /opt/coiled/env/.path-deps/xarray-2025.3.2.dev31+gd19619b4-py3   │
│           │ -none-any.whl                                                    │
│           ╵                                                                  │
╰──────────────────────────────────────────────────────────────────────────────╯

Output()

2025-04-29 05:31:00,823 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=10 maximum=200


## This takes ~10 minutes

In [43]:
pixel_counts = pixel_counts.persist()

/opt/coiled/env/lib/python3.12/site-packages/distributed/client.py:3357: UserWarning: Sending large graph of size 77.38 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [44]:
pixel_counts.load()

<xarray.Dataset> Size: 11kB
Dimensions:      (year: 5, lclu_bins: 13, adm_0: 21)
Coordinates:
  * year         (year) int64 40B 2000 2005 2010 2015 2020
    spatial_ref  int64 8B 0
  * lclu_bins    (lclu_bins) object 104B 'Terra Firma True desert' ... 'Ocean'
  * adm_0        (adm_0) object 168B 'ARG' 'BOL' 'BRA' ... 'TTO' 'URY' 'VEN'
Data variables:
    lclu         (year, lclu_bins, adm_0) float64 11kB 2.499e+08 ... 7.42e+03
Attributes:
    license:  CC BY 4.0

In [45]:
pixel_counts.to_netcdf("demo-pixels.nc")

## Visualize

In [46]:
pixel_counts.to_dataframe()

lclu  spatial_ref
year lclu_bins               adm_0                          
2000 Terra Firma True desert ARG    249948446.0            0
                             BOL     63320932.0            0
                             BRA      1022613.0            0
                             CHL    352977180.0            0
                             COL        75717.0            0
...                                         ...          ...
2020 Ocean                   SGS            NaN            0
                             SUR          419.0            0
                             TTO            NaN            0
                             URY       301504.0            0
                             VEN         7420.0            0

[1365 rows x 2 columns]

In [48]:
cluster.shutdown()

2025-04-29 05:40:33,654 - distributed.deploy.adaptive - INFO - Adaptive scaling stopped: minimum=10 maximum=200. Reason: unknown
